In [ ]:
from tqdm import tqdm
import pandas as pd
from sklearn import metrics
from scipy.spatial.distance import cdist

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 163 kB 88.9 MB/s 
     |████████████████████████████████| 7.6 MB 89.1 MB/s 


In [ ]:
!pip install parallelformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 48 kB 557 kB/s eta 0:00:011
  Created wheel for parallelformers: filename=parallelformers-1.2.7-py3-none-any.whl size=117791 sha256=ce1c96f5d462c55210041d65abcc897059cc1349cfafe5afcc489f60b6fbb7c6
  Stored in directory: /home2/sashank.sridhar/.cache/pip/wheels/4f/19/42/8d74380c84a1e93401ee163f3bd545853051a4d895fd95ca2e
Successfully built parallelformers


In [ ]:
!pip install faiss-gpu

Download snomed term-concept file from UMLS website

In [ ]:
snomed_csv = pd.read_csv("data/sct2_Description_Snapshot-en_INT_20220831.txt", delimiter="\t")
print(len(snomed_csv))

1569232


In [ ]:
snomed_csv.columns

Index(['id', 'effectiveTime', 'active', 'moduleId', 'conceptId',
       'languageCode', 'typeId', 'term', 'caseSignificanceId'],
      dtype='object')

In [ ]:
snomed_csv.head()

,id,effectiveTime,active,moduleId,conceptId,languageCode,typeId,term,caseSignificanceId
0,101013,20170731,1,900000000000207008,126813005,en,900000000000013009,Neoplasm of anterior aspect of epiglottis,900000000000448009
1,102018,20170731,1,900000000000207008,126814004,en,900000000000013009,Neoplasm of junctional region of epiglottis,900000000000448009
2,103011,20170731,1,900000000000207008,126815003,en,900000000000013009,Neoplasm of lateral wall of oropharynx,900000000000448009
3,104017,20170731,1,900000000000207008,126816002,en,900000000000013009,Neoplasm of posterior wall of oropharynx,900000000000448009
4,105016,20170731,1,900000000000207008,126817006,en,900000000000013009,Neoplasm of esophagus,900000000000448009


Process snomed terms

In [ ]:
all_ids = snomed_csv['conceptId']
all_names = []
for i in snomed_csv['term']:
    try:
        all_names.append(i.lower())
    except:
        all_names.append('not applicable')
#         print(i)

In [ ]:
snomed_csv.iloc[906846]

id                            1491117014
effectiveTime                   20030131
active                                 1
moduleId              900000000000207008
conceptId                      385432009
languageCode                          en
typeId                900000000000013009
term                                 NaN
caseSignificanceId    900000000000020002
Name: 906846, dtype: object

In [ ]:
snomed_name_id = [(all_names[i], all_ids[i]) for i in range(len(all_ids))]

In [ ]:
len(all_ids)

1569232

In [ ]:
all_names[:10]

['neoplasm of anterior aspect of epiglottis',
 'neoplasm of junctional region of epiglottis',
 'neoplasm of lateral wall of oropharynx',
 'neoplasm of posterior wall of oropharynx',
 'neoplasm of esophagus',
 'neoplasm of cervical esophagus',
 'neoplasm of thoracic esophagus',
 'neoplasm of abdominal esophagus',
 'neoplasm of middle third of esophagus',
 'neoplasm of lower third of esophagus']

In [ ]:
all_ids[:10]

0    126813005
1    126814004
2    126815003
3    126816002
4    126817006
5    126818001
6    126819009
7    126820003
8    126822006
9    126823001
Name: conceptId, dtype: int64

# load sapbert

In [ ]:
import numpy as np
import torch

In [ ]:
GPU_COUNT = torch.cuda.device_count()

In [ ]:
GPU_COUNT

4

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") ## specify the GPU id's, GPU id's start from 0.
device

device(type='cuda')

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")

2022-11-03 15:18:28.534264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 15:18:28.796577: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-03 15:18:30.642085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/opt/cudnn-7.6.5.32-cuda-10.2/lib64
2022-11-03 15:18:30.642256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; 

In [ ]:
# model = torch.nn.DataParallel(model)
# model.to(device)
from parallelformers import parallelize
parallelize(model, num_gpus=4, fp16=True)

2022-11-03 15:18:42.016139: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 15:18:42.048513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 15:18:42.083803: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

Generate embeddings for snomed labels

In [ ]:
# all_names1 = all_names[:100]

In [ ]:
bs = 128
all_reps = []
for i in tqdm(np.arange(0, len(all_names), bs)):
    toks = tokenizer.batch_encode_plus(all_names[i:i+bs],
                                      padding="max_length",
                                      max_length=25,
                                      truncation=True,
                                      return_tensors="pt")
    toks = toks.to(device)
    output = model(**toks)
    cls_rep = output[0][:,0,:]
    
    all_reps.append(cls_rep.cpu().detach().numpy())
all_reps_emb = np.concatenate(all_reps, axis=0)

100%|█████████████████████████████████████| 12260/12260 [17:08<00:00, 11.92it/s]


In [ ]:
print(all_reps_emb.shape)

(1569232, 768)


In [ ]:
all_reps_emb = all_reps_emb.astype(np.float32)

In [ ]:
import faiss

In [ ]:
d = all_reps_emb.shape[1]
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)

True


In [ ]:
index.add(all_reps_emb)                  # add vectors to the index
print(index.ntotal)

1569232


Load ground truth data

In [ ]:
entities = pd.read_csv("data/entities.tsv", delimiter="\t")
entities.head()

,filename,mark,label,offset1,offset2,span,code
0,es-S0212-71992007000100007-1,T1,ENFERMEDAD,40,61,arterial hypertension,38341003
1,es-S0212-71992007000100007-1,T2,ENFERMEDAD,66,79,polyarthrosis,36186002
2,es-S0212-71992007000100007-1,T3,ENFERMEDAD,1682,1698,pleural effusion,60046008
3,es-S0212-71992007000100007-1,T4,ENFERMEDAD,1859,1875,pleural effusion,60046008
4,es-S0212-71992007000100007-1,T5,ENFERMEDAD,1626,1648,lower lobe atelectasis,46621007


In [ ]:
inp_names = [i.lower() for i in entities['span']]
inp_labels = entities['code']
print(inp_names[:10])
print(inp_labels[:10])

['arterial hypertension', 'polyarthrosis', 'pleural effusion', 'pleural effusion', 'lower lobe atelectasis', 'infectious spondylodiscitis d10-d11', 'pleural effusion', 'brucellosis', 'orchiepididymitis', 'goitre']
0     38341003
1     36186002
2     60046008
3     60046008
4     46621007
5    302935008
6     60046008
7     75702008
8    197983000
9      3716002
Name: code, dtype: object


In [ ]:
# c=0
# for i in inp_label:
# #     if type(i)!=float:
#     try:
#         [float(i)]
#     except:
#         c+=1
# #         print(i.split('+'))
# c

In [ ]:
# inp_names1 = inp_names[:10]

Generate embeddings for ground truth terms, get their closest snomedct embedding and list out its corresponding snomedct code

In [ ]:
query_toks = tokenizer.batch_encode_plus(list(inp_names),
                                        padding = "max_length",
                                        max_length = 25,
                                        truncation=True,
                                        return_tensors="pt")
query_toks = query_toks.to(device)
query_output = model(**query_toks)
query_cls_rep = query_output[0][:,0,:]

In [ ]:
query_cls_rep = query_cls_rep.cpu().detach().numpy()

In [ ]:
query_cls_rep = query_cls_rep.astype(np.float32)

In [ ]:
k= 1 # take the 1 closest neighbor

In [ ]:
D, I = index.search(query_cls_rep, k)

In [ ]:
print(I[:5])

[[473409]
 [ 58583]
 [ 96888]
 [ 96888]
 [477684]]


In [ ]:
pred_ids = [all_ids[i[0]] for i in I]
# score=sum((pred_ids[i]==inp_label[i])*1 for i in range(len(pred_ids)))
# score/len(inp_label)

In ground truth, zero or more than one codes are also present for each term; here only one code is predicted for each query

In [ ]:
p = [[i] for i in pred_ids]
t = []
for i in inp_labels:
    try:
        t.append([int(i)])
    except:
        try:
            t.append([int(j) for j in (i.split('+'))])
        except:
#             print('nomap')
            t.append([])


In [ ]:
p[0][0]

38341003

In [ ]:
p[0][0] in t[0]

True

In [ ]:
pre = 0
for i in range(len(p)):
    if p[i][0] in t[i]:
        pre+=1

pre /= len(p)
print('precision', pre)


rec = 0
for i in range(len(t)):
    if len(t[i])==1:
        if t[i][0] in p[i]:
            rec+=1
    elif len(t[i])>1:
        for j in range(len(t[i])):
            if t[i][j] in p[i]:
                rec+=1

rec /= sum(len(i) for i in t)
print('recall', rec)       


f1 = 2*pre*rec/(pre+rec+np.finfo(np.float32).eps)
print('f1', f1)

precision 0.3675187969924812
recall 0.359147685525349
f1 0.3632849645578643
